In [ ]:
import csv
import glob
import json
import os
import sys
import urllib.request

## --- parameter ---
CONTEST = "W398" # Contest name
CONTEST_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-398" # API URL

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

In [ ]:
# create folder
try:
    os.mkdir(CONTEST)
    os.mkdir(CONTEST+"/Q3")
    os.mkdir(CONTEST+"/Q4")
    os.mkdir(CONTEST+"/Q3/0001-1000")
    os.mkdir(CONTEST+"/Q3/1001-2000")
    os.mkdir(CONTEST+"/Q3/2001-3000")
    os.mkdir(CONTEST+"/Q3/3001-4000")
    os.mkdir(CONTEST+"/Q3/4001-5000")
    os.mkdir(CONTEST+"/Q3/5001-6000")
    os.mkdir(CONTEST+"/Q4/0001-1000")
    os.mkdir(CONTEST+"/Q4/1001-2000")
    os.mkdir(CONTEST+"/Q4/2001-3000")
    os.mkdir(CONTEST+"/Q4/3001-4000")
    os.mkdir(CONTEST+"/Q4/4001-5000")
    os.mkdir(CONTEST+"/Q4/5001-6000")
except:
    pass

os.mkdir(CONTEST+"/Q3/6001-7000")
os.mkdir(CONTEST+"/Q3/7001-8000")
os.mkdir(CONTEST+"/Q4/6001-7000")
os.mkdir(CONTEST+"/Q4/7001-8000")

In [ ]:
## get question_id
for k in range(10):
    try:
        req = urllib.request.Request(CONTEST_URL, headers=headers)
        r = urllib.request.urlopen(req).read()
        response = json.loads(r)
    except:
        pass
    else:
        break
else:
    print("error")
    sys.exit()
    
for question in response["questions"]:
    print(question["question_id"], question["credit"], question["title"])

In [ ]:
## create the csv to record user name and submission_id

## --- parameter ---
PAGE = 299 # how many page gonna fetch
Q = ["3416", "3414"] # [Q3, Q4], use question_id get from above

with open(CONTEST+"/"+CONTEST+".csv", "w") as f:
    f.write("user, Q3, Q4\n");
    
    for pagination in range(1, PAGE+1):
        for k in range(10):
            try:
                req = urllib.request.Request(CONTEST_URL+"?pagination=" + str(pagination) +"&region=global", headers=headers)
                r = urllib.request.urlopen(req).read()
                response = json.loads(r)
            except:
                pass
            else:
                break
        else:
            print(pagination)
            sys.exit()

        print(pagination)

        for i in range(25):
            rank = (pagination-1)*25 + i + 1
            
            f.write(response["total_rank"][i]["user_slug"])
            
            for j, Q_id in enumerate(Q):
                if Q_id not in response["submissions"][i]:
                    submission_id = ""
                else:   
                    submission_id = str(response["submissions"][i][Q_id]["submission_id"])
    
                f.write("," + submission_id)

            f.write("\n")
    
print("finished!!!")

In [ ]:
## download code
def lang_ext(lang):
    match lang:
        case "cpp":
            return "cpp"
        case "python3" | "python":
            return "py"
        case "java":
            return "java"
        case _:
            return lang

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))

    for rank, row in enumerate(rows):
        if rank <= 0:
            continue
            
        folder = str(rank//1000)+"001-"+str(rank//1000+1)+"000/"
        
        if rank % 25 == 0: # verbose
            print(rank)
        
        if row[1] and not glob.glob(CONTEST+"/Q3/"+folder+str(rank)+".*"):
            submission_id = row[1]
            for k in range(20):
                try:
                    req = urllib.request.Request("https://leetcode.com/api/submissions/" + submission_id, headers=headers)
                    r = urllib.request.urlopen(req).read()
                    response2 = json.loads(r)
                except:
                    pass
                else:
                    with open(CONTEST+"/Q3/"+folder+str(rank)+"."+lang_ext(response2["lang"]), "w", encoding="utf-8") as f2:
                        try:
                            f2.write(response2["code"])
                        except:
                            print(submission_id)
                            sys.exit()
                    break
            else:
                print("file missed", rank, "Q3", submission_id)

        if row[2] and not glob.glob(CONTEST+"/Q4/"+folder+str(rank)+".*"):            
            submission_id = row[2]
            for k in range(20):
                try:
                    req = urllib.request.Request("https://leetcode.com/api/submissions/" + submission_id, headers=headers)
                    r = urllib.request.urlopen(req).read()
                    response2 = json.loads(r)
                except:
                    pass
                else:
                    with open(CONTEST+"/Q4/"+folder+str(rank)+"."+lang_ext(response2["lang"]), "w", encoding="utf-8") as f2:
                        try:
                            f2.write(response2["code"])
                        except:
                            print(submission_id)
                            sys.exit()
                    break
            else:
                print("file missed", rank, "Q4", submission_id)
                

print("finished!!!")